In [1]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import re

In [2]:
raw = requests.get("https://www.fci.be/en/statistics/")
web = bs(raw.text, "html.parser")


In [3]:
submenu = web.find('div', {'class':'submenu'})

links = []
for i in submenu.find_all('a')[2:10]:
    links.append("https://www.fci.be/en/statistics/" + i['href'])

In [4]:
links

['https://www.fci.be/en/statistics/ByYear.aspx?year=2021',
 'https://www.fci.be/en/statistics/ByYear.aspx?year=2020',
 'https://www.fci.be/en/statistics/ByYear.aspx?year=2019',
 'https://www.fci.be/en/statistics/ByYear.aspx?year=2018',
 'https://www.fci.be/en/statistics/ByYear.aspx?year=2017',
 'https://www.fci.be/en/statistics/ByYear.aspx?year=2016',
 'https://www.fci.be/en/statistics/ByYear.aspx?year=2015',
 'https://www.fci.be/en/statistics/ByYear.aspx?year=2014']

In [12]:
dfs = []
years = []

In [13]:
for i in links:
    r = requests.get(i)
    soup = bs(r.text, 'html.parser')
    year_title = soup.find('h1').text.strip()
    year = re.search('\d{4}', year_title).group()
    years.append(year)
    
    headers = []
    table_header = soup.find('table', attrs={'class': "header"})
    for i in table_header.find_all('td'):
        headers.append(i.text.strip())
    
    table = soup.find('table', attrs={'id': 'ContentPlaceHolder1_StatistiquesGridView'})
    table_rows = table.find_all('tr')
    l = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.get_text(strip=True) for tr in td]
        l.append(row)
    df = pd.DataFrame(l, columns = headers)
    df['year'] = year
    dfs.append(df)
    
    

In [18]:
for i, j in zip(dfs, years):
    i.to_csv(j + '.csv', index = False)

In [8]:
years

['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014']

In [19]:
all_df = pd.concat(dfs)

In [22]:
all_df.to_csv('all_years.csv', index = False)

## Data Cleaning

In [60]:
dfs[1].dtypes

                                                                                  object
PUPPIES registered with the studbook and the appendix                             object
LITTERS registered with the studbook and the appendix                             object
CAC ALL-BREED SHOWS                                                               object
CACIB ALL-BREED SHOWS                                                             object
FCI (breed, group, all-breed) JUDGES                                              object
INDIVIDUAL MEMBERS (people)                                                       object
member (breed, regional, sporting, etc.) CLUBS                                    object
PURE-BRED DOGS REGISTERED WITH THE STUDBOOK                                       object
TOTAL number of PURE-BRED DOGS REGISTERED WITH THE STUDBOOK since its creation    object
Approximate TOTAL number of DOGS (PURE-BRED OR NOT)                               object
Approximate TOTAL num

In [61]:
dfs[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                                                                          Non-Null Count  Dtype 
---  ------                                                                          --------------  ----- 
 0                                                                                   100 non-null    object
 1   PUPPIES registered with the studbook and the appendix                           100 non-null    object
 2   LITTERS registered with the studbook and the appendix                           100 non-null    object
 3   CAC ALL-BREED SHOWS                                                             100 non-null    object
 4   CACIB ALL-BREED SHOWS                                                           100 non-null    object
 5   FCI (breed, group, all-breed) JUDGES                                            100 non-null    object
 6   INDIVIDUAL MEMBERS (people)

In [66]:
new_df = dfs[1]

In [70]:
new_df['PUPPIES registered with the studbook and the appendix'].isna()

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: PUPPIES registered with the studbook and the appendix, Length: 100, dtype: bool

In [71]:
new_df.iloc[1]

                                                                                  ALGERIA
PUPPIES registered with the studbook and the appendix                                    
LITTERS registered with the studbook and the appendix                                    
CAC ALL-BREED SHOWS                                                                      
CACIB ALL-BREED SHOWS                                                                    
FCI (breed, group, all-breed) JUDGES                                                     
INDIVIDUAL MEMBERS (people)                                                              
member (breed, regional, sporting, etc.) CLUBS                                           
PURE-BRED DOGS REGISTERED WITH THE STUDBOOK                                              
TOTAL number of PURE-BRED DOGS REGISTERED WITH THE STUDBOOK since its creation           
Approximate TOTAL number of DOGS (PURE-BRED OR NOT)                                      
Approximat

In [72]:
for i in new_df.iloc[1]:
    print(i)

ALGERIA











2020


In [77]:
ll = new_df['PUPPIES registered with the studbook and the appendix'].str.strip()

In [79]:
ll.astype(bool)

0     False
1     False
2     False
3      True
4      True
      ...  
95     True
96     True
97     True
98    False
99    False
Name: PUPPIES registered with the studbook and the appendix, Length: 100, dtype: bool

In [83]:
ll.str.isdigit()

0     False
1     False
2     False
3      True
4     False
      ...  
95     True
96    False
97     True
98    False
99    False
Name: PUPPIES registered with the studbook and the appendix, Length: 100, dtype: bool